In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings

from dataprocess import dataprocessor as dp
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame

with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    from keras.models import load_model
    from nn_source import models as mp
    from rl_source import alumnienv

Using TensorFlow backend.


In [2]:
# read the pickled file for ahu data
dfdata = dp.readfile('../data/processed/buildingdata.pkl')

# return pickled df
df = dfdata.return_df(processmethods=['file2df'])

# 0-1 Scale the dataframe before sending it to the environment
scaler = MinMaxScaler(feature_range=(0,1))
dfscaler = scaler.fit(df)
dfscaled = DataFrame(dfscaler.fit_transform(df), index=df.index, columns=df.columns)

In [4]:
# load energy model
energymodel = load_model('../results/lstmtrain/LSTM_model_44_0.01');  # Specify the path where energy is stored

In [5]:
# important parameters to scale the reward
params = {
    'energy_saved': 1.0,
    'energy_savings_thresh': 0.0,
    'energy_penalty': -1.0,
    'energy_reward_weight': 1.0,
    'comfort': 1.0,
    'comfort_thresh': 0.10,
    'uncomfortable': -1.0,
    'comfort_reward_weight': 1.0,
    'action_minmax': [np.array([df.sat.min()]),
                      np.array([df.sat.max()])]
}

In [6]:
# instantiate the environment
envmodel = alumnienv.Env(
    df=dfscaled,
    obs_space_vars=['oat', 'orh', 'ghi', 'sat', 'avg_stpt'],  # state space variable
    action_space_vars=['sat'],   # action space variable
    action_space_bounds=[[-2.0], [2.0]],  # bounds for real world action space; is scaled internally using the params
    energy_model=energymodel,  # trained lstm model
    model_input_shape=(1, 1, 4),  # lstm model input data shape (no_samples, output_timestep, inputdim)
    model_input_vars=['oat', 'orh', 'ghi','sat'],  # lstm model input variables
    **params
)

/data/nauga/SmartBuildings/venvs/sbvenv1/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Now we pass these elements to vecotrized environments needed for PPO

In [7]:
os.cpu_count()

20